# Create Concatenated Datasets

In this notebook, we create video clips to evaluate **mexca**'s perfomance on face identification. We select shots from the Dutch election debate 2021 and concatenate them to a new video file. For each of the 15 different faces in the video, shots that contain that contain the face are selected until a maximum number of seconds is reached. Thus, every face is presented approximately equally (i.e, in the same number of frames) to simulate an ideal condition for unsupervised face identification.

For the maximum number of seconds each face is presented, we choose 30s, 60s, and 180s to get an impression on how the duration influences the identification performance. We expect that longer duration should lead to better performance.

In [1]:
import os
import json
import numpy as np
import pandas as pd
from moviepy.editor import VideoFileClip, concatenate_videoclips

In [2]:
# Static variables
ANNOTATION_PATH = 'camera_shots_coding.csv'
PREFIX = os.path.join('datasets', 'debate_conc_shots')
NUM_LABELS = 15
MAX_DURATION = (30.0, 60.0, 180.0)

In [3]:
def cut_time(time):
    """Strip milliseconds from time format.
    """
    time_split = str(time).split(":")
    out = ':'.join(time_split[:-1])
    return out

In [4]:
def format_time_cols(df):
    """Create and format time related columns in data frame.
    """
    df['start_shot'] = pd.to_timedelta(df['begin_frame'].apply(cut_time))
    df['duration'] = np.abs(df['start_shot'].diff(periods=-1))
    df['end_shot'] = df['start_shot'] + df['duration']

    return df

In [5]:
def read_annotation_df(filepath):
    """Read and format annotation csv.
    """
    annotation_df = format_time_cols(
        pd.read_csv(filepath, sep=';')
    )

    return annotation_df


In [6]:
def get_unique_labels(df):
    """Get unique face labels from data frame. Remove 'nan' and 'group'.
    """
    unique_labels = np.unique(
        df.face_1.unique().tolist() + 
        df.face_2.unique().tolist() + 
        df.face_3.unique().tolist()
    )
    unique_labels = [l for l in unique_labels if l not in ['nan', 'group']]

    return unique_labels

In [7]:
def select_shots(df, max_duration, num_labels):
    """Select shots for data set until for each face label the total duration of shots amounts to `max_duration`.
    """
    unique_labels = get_unique_labels(df)[:(num_labels + 1)]

    labels_duration = {}

    for label in unique_labels:
        labels_duration[label] = []

    selected_shot_names = {}

    for row_tuple in df.itertuples():
        faces = (row_tuple.face_1, row_tuple.face_1, row_tuple.face_1)
        filename = row_tuple.filename

        # Check if shot is already selected
        if filename not in selected_shot_names:
            for face in faces:
                # Check if face is selected and whether the total duration is below maximum
                if str(face) in unique_labels and np.sum(labels_duration[face]) < max_duration:
                    selected_shot_names[row_tuple.shot_id] = filename
                    # Add duration of shot to record
                    labels_duration[face].append(row_tuple.duration.total_seconds())

    for label in labels_duration:
        print(label, np.sum(labels_duration[label]))

    return selected_shot_names

In [8]:
def get_shot_filepaths(shot_names):
    """Get the relative filepaths of selected shots.
    """
    filenames = []

    for shot in shot_names:
        # Get batch folder number
        batch = int(np.ceil(shot/50))
        filename = shot_names[shot] + '.mp4'
        path = os.path.join('shots', f'batch{batch}', filename)
        # Check if filepath exists
        exists = os.path.exists(path)
        if exists:
            filenames.append(path)
        else:
            print(shot, batch, filename, path)

    return filenames

In [9]:
def create_video_clip(filenames, outname):
    """Create a video clip by concatenating selected shots.
    """
    clips = []

    for filename in filenames:
        clip = VideoFileClip(filename)
        clips.append(clip)

    conc_clip = concatenate_videoclips(clips)

    if not os.path.exists(outname):
        conc_clip.write_videofile(outname)

    return conc_clip.fps


In [10]:
def create_annotation_file(df, shot_names, fps, outname):
    """Create an annotation csv for the concatenated shots.
    """
    conc_annotation = {
        'shot_id': [],
        'shot_name': [],
        'frame': [],
        'time': [],
        'face_label': []
    }

    frame = 0
    start_t = 0.0

    for row_tuple in df.itertuples():
        shot_name = row_tuple.filename

        # Check if shot is selected
        if shot_name in shot_names.values():
            shot_id = row_tuple.shot_id
            dur = row_tuple.duration.total_seconds()
            # Create new timestamps
            times = np.linspace(start_t, start_t+dur, int(dur*fps))
            labels = (row_tuple.face_1, row_tuple.face_2, row_tuple.face_3)
            for i, t in enumerate(times):
                for label in labels:
                    if not pd.isna(label):
                        conc_annotation['shot_id'].append(shot_id)
                        conc_annotation['shot_name'].append(shot_name)
                        conc_annotation['frame'].append(frame + i)
                        conc_annotation['time'].append(round(t, 3))
                        conc_annotation['face_label'].append(label)

            # Update timestamp and frame
            start_t += dur
            frame += int(dur*fps)

    # Export to csv
    pd.DataFrame(conc_annotation).to_csv(outname)

In [11]:
def create_dataset_from_selected_shots(filepath, prefix, max_duration, num_labels):
    annotation_df = read_annotation_df(filepath)

    selected_shot_names = select_shots(
        annotation_df,
        max_duration,
        num_labels
    )

    filenames = get_shot_filepaths(selected_shot_names)

    fps = create_video_clip(
        filenames,
        prefix + f'_{num_labels}spk_{int(max_duration)}s.mp4'
    )

    create_annotation_file(
        annotation_df,
        selected_shot_names,
        fps,
        prefix + f'_{num_labels}spk_{int(max_duration)}s.csv'
    )


In [12]:
def write_metadata_file(prefix, max_duration, num_labels):
    """Link video clips and annotations in metadata json file.
    """
    metadata = {}

    for dur in max_duration:
        metadata[str(dur)] = {
            'video': prefix + f'_{num_labels}spk_{int(dur)}s.mp4',
            'annotation': prefix + f'_{num_labels}spk_{int(dur)}s.csv'
        }

    with open(prefix + '_datasets.json', 'w') as file:
        json.dump(metadata, file)

In [13]:
for dur in MAX_DURATION:
    create_dataset_from_selected_shots(ANNOTATION_PATH, PREFIX, dur, NUM_LABELS)

Hoekstra 30.0
Kaag 30.0
Klaver 33.0
Marijnissen 33.0
Rutte 30.0
Wilders 30.0
citizen1 37.0
citizen2_f 30.0
citizen2_m 21.0
citizen3 30.0
citizen4 42.0
citizen5 36.0
citizen6 33.0
mod_f 40.0
mod_m 33.0
Moviepy - Building video datasets\debate_conc_shots_15spk_30s.mp4.
MoviePy - Writing audio in debate_conc_shots_15spk_30sTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video datasets\debate_conc_shots_15spk_30s.mp4



Moviepy - Done !
Moviepy - video ready datasets\debate_conc_shots_15spk_30s.mp4
Hoekstra 86.0
Kaag 60.0
Klaver 66.0
Marijnissen 60.0
Rutte 60.0
Wilders 63.0
citizen1 60.0
citizen2_f 60.0
citizen2_m 21.0
citizen3 66.0
citizen4 61.0
citizen5 63.0
citizen6 61.0
mod_f 66.0
mod_m 62.0
Moviepy - Building video datasets\debate_conc_shots_15spk_60s.mp4.
MoviePy - Writing audio in debate_conc_shots_15spk_60sTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video datasets\debate_conc_shots_15spk_60s.mp4



Moviepy - Done !
Moviepy - video ready datasets\debate_conc_shots_15spk_60s.mp4
Hoekstra 180.0
Kaag 186.0
Klaver 180.0
Marijnissen 180.0
Rutte 189.0
Wilders 180.0
citizen1 183.0
citizen2_f 180.0
citizen2_m 21.0
citizen3 180.0
citizen4 180.0
citizen5 183.0
citizen6 182.0
mod_f 180.0
mod_m 180.0
Moviepy - Building video datasets\debate_conc_shots_15spk_180s.mp4.
MoviePy - Writing audio in debate_conc_shots_15spk_180sTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video datasets\debate_conc_shots_15spk_180s.mp4



Moviepy - Done !
Moviepy - video ready datasets\debate_conc_shots_15spk_180s.mp4


In [14]:
write_metadata_file(PREFIX, MAX_DURATION, NUM_LABELS)